# Introduction
This file extracts information from the wildfire dataset about fires and their distance from a target city. This will likely take over an hour to run.

### Output
This file creates an output of 
* 'data512_common_analysis/data/fires.json
    > This file will contain the fire year, fire type, acres burned,hectares burned, and distance from the target city for every fire in the dataset. The target city is Pullman, WA for this file.
    
### Usage 
Due to the size of the dataset, you will need to download the source file from the USGS website [here](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) by clicking on "GeoJSON Files.zip"
Once you have this folder downloaded, unzip it and either change the 'f' variable below to point to your export location or move the 'USGS_Wildland_Fire_Combined_Dataset.json' file into the 'GeoJSON_Exports' folder in this repository. The file should be a little over 3GB (as of 2023).

### Acknowledgment
This is adapted from the work of Dr. David W. McDonald whose source code can be found [here](https://colab.research.google.com/drive/1qNI6hji8CvDeBsnLDAhJXvaqf2gcg8UV#scrollTo=3dEmQWT5f1z5). This is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/)

### License
This code example was developed by Chandler Ault for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). 


In [13]:
from pandas_geojson import read_geojson
from pyproj import Transformer, Geod
import json

In [5]:
# Change this to point to your file or move the file into the GeoJSON_Exports folder
f = './GeoJSON_Exports/USGS_Wildland_Fire_Combined_Dataset.json'

In [6]:
# Reads the file using geojson
import geojson
with open(f) as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [39]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

In [ ]:
# Extract fires and their distance from a target city. Add the necessary information of a new city to CITY_LOCATIONS and change the place variable to point to that city
# to get fires adjacent to other cities.
fires = []
CITY_LOCATIONS = {
    'pullman' :       {'city'   : 'Pullman',
                       'county' : 'Whitman',
                       'state'  : 'Washington',
                       'fips'   : '53075',
                       'latlon' : [46.7298, -117.1817] }, 
   }
place = CITY_LOCATIONS['pullman']['latlon']
for i in range(0,len(gj)):
    try:
        if i%1000 == 0:
            print(i)
        
        ring = gj[i]['geometry']['rings'][0]

        selected_keys = ['USGS_Assigned_ID', 'Fire_Year', 'Assigned_Fire_Type', 'GIS_Acres', 'GIS_Hectares']
        info ={key: gj[i]['attributes'][key] for key in selected_keys}
        info['Distance'] = shortest_distance_from_place_to_fire_perimeter(place, ring)[0]
        fires.append(info)
    except:
         print('failed')
    
with open('../data/fires.json', 'w') as file:
            json.dump(fires, file)